# Decision Tree
This Notebook features an implementation of a simple binary decision tree from scratch.

### Problem:
- Build a decision tree.
- Your tree should have the variable height.
- For each dataset plot the dependence of classification accuracy on the height of the tree.


### Implementation features:
- Gini coefficient as information gain measure
- Variable tree depth
- Optimization of tree depth for given data up to the maximum depth
- Basic metrics and a data_prep class to preprocess the data


### Important functions: 
- decision_tree.find_tree() : finds the optimal tree for given data and given max depth 
- decision_tree.class.predict() : predicts new data and outputs accuracy and confusion


In [1]:
import random 
import numpy
import math 
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# data preperation class 
class data_prep:                                 #Data preparation and normalization for the model 
    def __init__(self,location):
        self.data = open(location, "r+")
        self.data = self.data.read()
        self.data = self.data.splitlines()
        for x in range(len(self.data)):
            self.data[x] = self.data[x].split(" ")
                       
    def transform(self,numbers = "y"):
        data = self.data
        Y = []
        del data[0]
        del data[0]
        for w in range(len(data)):
            for y in range(len(data[w])):
                data[w][y] = float(data[w][y]) 
        features = len(data[0])-1
        for x in range(len(data)):
            Y.append(data[x][features:])
            data[x] = data[x][:features]
        self.transformedX = numpy.array(data)
        self.transformedY = (numpy.array(Y)).astype(int)
        
    def normalize(self):
        features = numpy.transpose(self.transformedX)
        toDelete = []
        for y in range(len(features)):
            maxi = max(features[y])
            mini = min(features[y])
            if maxi == mini: 
               print("Feature " + str(y) + " was spotted to be useless (Only 1 possibe value). It will be removed from the dataset")
               toDelete.append(y)
            else: 
                for z in range (len(features[y])):
                    features[y][z] =  (features[y][z]- mini)/ (maxi - mini)
        newFeatures = []
        for z in range(len(features)): 
            if z not in toDelete: 
                       newFeatures.append(features[z])
        newFeatures = numpy.array(newFeatures)  
        newFeatures = numpy.transpose(newFeatures)
        self.normalizedY = self.transformedY
        self.normalizedX = newFeatures               

In [131]:
#Data prep:

train = data_prep("data/tree/txt/01_train.txt")            
train.transform()      
train.normalize()
print(len(train.normalizedX))


test = data_prep("data/tree/txt/01_test.txt")
test.transform()
test.normalize()

3891


In [240]:
class decision_tree: 
    def __init__(self,X,Y, depth=10):   #initialize useful values and data
        self.n = len(X)
        self.X = X[int(self.n/20):]
        self.X_val = X[:int(self.n/20)]
        self.Y = Y[int(self.n/20):].T
        self.Y_val = Y[:int(self.n/20)].T
        self.n = len(self.X)
        self.classes = max(self.Y.T)[0]
        self.features = len(self.X.T)
        self.maxDepth = depth
        
    def gini(self,dataset):    #calculate the gini coefficient for a dataset
        count = {}
        for x in range (self.classes):
            count[x+1] = 0
        for y in range (len(dataset)):
            count[dataset[y]]+=1
        gini = 0 
        for z in range (self.classes):
            gini += count[z+1]/len(dataset)* (1-count[z+1]/len(dataset))
        return gini 
    
    def gini_gain(self,split1,split2,originalLen): #calculate gini gain for 2 datasets
        return self.gini(split1)*(len(split1)/originalLen) + self.gini(split2)*(len(split2)/originalLen)
       
    
    def find_best_split(self,labels,data):  #searches for the best split rule for dataset
        best_gain = 0 
        best_question = None
        current_uncertainty = self.gini(labels)
        for x in range (self.features):
            for y in range(1,100): 
                split1 = []
                split2 = []
                for z in range(0,len(data)):
                    if data[z][x] <= (y/100):
                        split1.append(labels[z])
                    else: 
                        split2.append(labels[z])
                if not split2: 
                    gain = 0
                elif not split1: 
                    gain = 0
                else:
                    gain = self.gini_gain(split1,split2,len(data))
                    gain = gain - current_uncertainty
                if gain < best_gain: 
                    best_gain = gain 
                    best_question = [x,y/100]
        return -1* best_gain ,best_question
    
    def get_ratio(self,label):      #helper function for leaf probabilities
        ratio = {}
        for x in label: 
            if x in ratio: 
                ratio[x] +=1 
            else: 
                ratio[x] = 1
        for key in ratio: 
            ratio[key] = ratio[key]/len(label)
        return ratio
    
    def split(self,label,data,criteria):    #split dataset based on a binary rule
        split1 = []
        split2 = []
        label1 = []
        label2 = []
        for x in range (len(data)):
            if data[x][criteria[0]] <= criteria[1]:
                split1.append(data[x])
                label1.append(label[x])
            else: 
                split2.append(data[x])
                label2.append(label[x])
        return numpy.array(split1), numpy.array(label1,ndmin=2), numpy.array(split2),numpy.array(label2,ndmin=2)
            
    def build_tree(self,label,data,depth):  #build the tree recursively from root
        depth += 1
        gain,question = self.find_best_split(label,data)
        if gain == 0 or depth == self.maxDepth: 
            return self.get_ratio(label)
        split1,label1, split2, label2 = self.split(label,data,question)
        left = self.build_tree(label1[0], split1,depth)
        right = self.build_tree(label2[0], split2,depth)
        return [question,left,right]
    
    def find_tree(self): #find the best tree depth based on validation data
        results = []
        for x in range(self.maxDepth):
            self.structure = self.build_tree(self.Y[0],self.X,x-1)
            predictions = []
            for x in range (len(self.X_val)):
                prediction = self.find_prediction(self.X_val[x])
                predictions.append(prediction)
            results.append(self.accuracy(self.Y_val[0],predictions))
        names = list(reversed(list(range(1, self.maxDepth+1))))
        print(pd.DataFrame(data=results,index=names, columns=["Accuracy for Tree depth"]),"\n")
        print("The best accuracy was found for a tree depth of " + str(self.maxDepth - results.index(max(results))))
        self.structure = self.build_tree(self.Y[0],self.X,results.index(max(results))-1)
        self.depth = self.maxDepth - results.index(max(results))
        
    def predict(self,data,label):    #predict new data based on a decision tree
        predictions = []
        for x in range (len(data)):
            prediction = self.find_prediction(data[x])
            predictions.append(prediction)
        accuracy = self.accuracy(label.T[0],predictions)
        print("Accuracy of the predictions: " + str(accuracy) + "%\n")
        confusion = self.confusion_matrix(label.T[0],predictions)
        print(pd.DataFrame(data=confusion),"\n")
    
    def find_prediction(self,data):   #predict one data example
        decision = 0
        tree = self.structure
        rule = tree[0]
        while type(tree) != dict:
            if data[rule[0]] <= rule[1]:
                tree = tree[1]
                if type(tree) == dict:
                    break
                rule = tree[0]
                
                rule = tree[0]
            else:
                tree = tree[2] 
                if type(tree) == dict:
                    break
                rule = tree[0]
        probable = 0
        for key in tree: 
            if tree[key] > probable:
                probable = tree[key]
                decision = key
        return decision
                   
    def accuracy(self,label,predictions):   #metrics for the results
        accuracy = 0
        for x in range(len(predictions)): 
            if predictions[x] == label[x]:
                accuracy +=1
        return accuracy/len(predictions)*100
    
    def confusion_matrix(self,label,predictions):
        conf = numpy.zeros((self.classes,self.classes), dtype=int)
        for x in range(len(predictions)): 
            conf[predictions[x]-1][label[x]-1] += 1
        return conf      

In [241]:
tree = decision_tree(train.normalizedX,train.transformedY,depth = 5)
tree.find_tree()


    Accuracy for Tree depth
10                92.783505
9                 93.298969
8                 93.814433
7                 94.329897
6                 95.360825
5                 95.360825
4                 95.876289
3                 96.391753
2                 84.020619
1                 58.247423 

The best accuracy was found for a tree depth of 3


In [244]:
tree.predict(test.normalizedX,test.transformedY)

Accuracy of the predictions: 73.502955538422%

     0   1   2   3   4   5   6   7   8    9     10   11  12  13  14  15   16  \
0   584   0   0   0   0   0   0   0   0   24    70  131   0   0   0  21    0   
1     0   0   0   0   0   0   0   0   0    0     0    0   0   0   0   0    0   
2     0   0   0   0   0   0   0   0   0    0     0    0   0   0   0   0    0   
3     0   0   0   0   0   0   0   0   0    0     0    0   0   0   0   0    0   
4     0   0   0   0   0   0   0   0   0    0     0    0   0   0   0   0    0   
5     0   0   0   0   0   0   0   0   0    0     0    0   0   0   0   0    0   
6     0   0   0   0   0   0   0   0   0    0     0    0   0   0   0   0    0   
7     0   0   0   0   0   0   0   0   0    0     0    0   0   0   0   0    0   
8     0   0   0   0   0   0   0   0   0    0     0    0   0   0   0   0    0   
9     0   0   0   0   0   0   0   0   0  311   144   91   0   0   0  25    0   
10    0   0   0   0   0   0   0   0   0    0  1292  218   0   0   0  34  